In [2]:
import pandas as pd
import warnings
from sqlalchemy import create_engine, inspect  
warnings.filterwarnings('ignore')

engine = create_engine('sqlite:///Chinook_Sqlite.sqlite')
inspector = inspect(engine)

table_names = inspector.get_table_names()
table_names

['Album',
 'Artist',
 'Customer',
 'Employee',
 'Genre',
 'Invoice',
 'InvoiceLine',
 'MediaType',
 'Playlist',
 'PlaylistTrack',
 'Track']

<h3> Upon examination of all the tables, key tables required for the analysis were identified and loaded into dataframes. </h3>

In [3]:
# Loading key tables into Pandas DataFrames
df_invoices = pd.read_sql_table('Invoice', engine)
df_invoice_items = pd.read_sql_table('InvoiceLine', engine)
df_customers = pd.read_sql_table('Customer', engine)
df_tracks = pd.read_sql_table('Track', engine)
df_albums = pd.read_sql_table('Album', engine)
df_artists = pd.read_sql_table('Artist', engine)
df_genres = pd.read_sql_table('Genre', engine)
df_employees = pd.read_sql_table('Employee', engine)

In [4]:
# Checking for missing values
print("Missing values in Invoices:")
print(df_invoices.isnull().sum())
print("\nMissing values in Invoice Items:")
print(df_invoice_items.isnull().sum())

Missing values in Invoices:
InvoiceId              0
CustomerId             0
InvoiceDate            0
BillingAddress         0
BillingCity            0
BillingState         202
BillingCountry         0
BillingPostalCode     28
Total                  0
dtype: int64

Missing values in Invoice Items:
InvoiceLineId    0
InvoiceId        0
TrackId          0
UnitPrice        0
Quantity         0
dtype: int64


<h3> The most likely reason for the missing values is that not all countries have states or postal codes in the same way the US does. To fix that, we replace the missing values with 'N/A' after verifying first if our assumption is true. </h3>



In [5]:
missing_state_df = df_invoices[df_invoices['BillingState'].isnull()]
print("Countries for invoices with missing BillingState:")
print(missing_state_df['BillingCountry'].value_counts())
print("\n")

Countries for invoices with missing BillingState:
BillingCountry
France            35
Germany           28
United Kingdom    21
Portugal          14
Czech Republic    14
India             13
Norway             7
Chile              7
Belgium            7
Spain              7
Sweden             7
Finland            7
Denmark            7
Poland             7
Austria            7
Hungary            7
Argentina          7
Name: count, dtype: int64




In [6]:
# Filling the missing values with a descriptive placeholder.

df_invoices['BillingState'].fillna('N/A (Not Applicable)', inplace=True)
df_invoices['BillingPostalCode'].fillna('N/A', inplace=True)

# Verify the fix
print("Missing values in Invoices AFTER cleaning:")
print(df_invoices.isnull().sum())

Missing values in Invoices AFTER cleaning:
InvoiceId            0
CustomerId           0
InvoiceDate          0
BillingAddress       0
BillingCity          0
BillingState         0
BillingCountry       0
BillingPostalCode    0
Total                0
dtype: int64


In [7]:
df_invoices

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,N/A (Not Applicable),Germany,70174,1.98
1,2,4,2009-01-02,Ullevålsveien 14,Oslo,N/A (Not Applicable),Norway,0171,3.96
2,3,8,2009-01-03,Grétrystraat 63,Brussels,N/A (Not Applicable),Belgium,1000,5.94
3,4,14,2009-01-06,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2009-01-11,69 Salem Street,Boston,MA,USA,2113,13.86
...,...,...,...,...,...,...,...,...,...
407,408,25,2013-12-05,319 N. Frances Street,Madison,WI,USA,53703,3.96
408,409,29,2013-12-06,796 Dundas Street West,Toronto,ON,Canada,M6J 1V1,5.94
409,410,35,2013-12-09,"Rua dos Campeões Europeus de Viena, 4350",Porto,N/A (Not Applicable),Portugal,N/A,8.91
410,411,44,2013-12-14,Porthaninkatu 9,Helsinki,N/A (Not Applicable),Finland,00530,13.86


In [8]:
# Transforming data for analysis
# Creating Fact and Dimension tables for analysis

# --- fact_sales ---
query_fact = """
SELECT
    il.InvoiceLineId as sales_id,
    i.InvoiceId as invoice_id,
    i.CustomerId as customer_id,
    il.TrackId as track_id,
    i.InvoiceDate as invoice_date,
    i.BillingCountry as country,
    il.UnitPrice as unit_price,
    il.Quantity as quantity,
    (il.UnitPrice * il.Quantity) as total_sale
FROM InvoiceLine il
INNER JOIN Invoice i ON il.InvoiceId = i.InvoiceId;
"""
fact_sales = pd.read_sql_query(query_fact, engine)
# Convert invoice_date to a proper datetime format
fact_sales['invoice_date'] = pd.to_datetime(fact_sales['invoice_date'])

# --- dim_tracks ---
query_dim_track = """
SELECT
    t.TrackId,
    t.Name as track_name,
    a.Title as album_title,
    ar.Name as artist_name,
    g.Name as genre_name,
    t.UnitPrice as track_price
FROM Track t
INNER JOIN Album a ON t.AlbumId = a.AlbumId
INNER JOIN Artist ar ON a.ArtistId = ar.ArtistId
INNER JOIN Genre g ON t.GenreId = g.GenreId;
"""
dim_tracks = pd.read_sql_query(query_dim_track, engine)

# ---dim_customers ---
query_dim_customer = """
SELECT
    c.CustomerId,
    c.FirstName || ' ' || c.LastName as customer_name,
    c.Company,
    c.City,
    c.Country,
    c.SupportRepId,
    e.FirstName || ' ' || e.LastName as support_rep_name
FROM Customer c
LEFT JOIN Employee e ON c.SupportRepId = e.EmployeeId;
"""
dim_customers = pd.read_sql_query(query_dim_customer, engine)

<h3>Exploratory data review using statistical summary</h3>

In [9]:
fact_sales.describe()

,sales_id,invoice_id,customer_id,track_id,invoice_date,unit_price,quantity,total_sale
count,2240.000000,2240.000000,2240.000000,2240.000000,2240,2240.000000,2240.0,2240.000000
mean,1120.500000,206.868750,29.974107,1717.734375,2011-06-25 13:42:12.857142784,1.039554,1.0,1.039554
min,1.000000,1.000000,1.000000,1.000000,2009-01-01 00:00:00,0.990000,1.0,0.990000
25%,560.750000,103.000000,15.000000,874.000000,2010-03-21 00:00:00,0.990000,1.0,0.990000
50%,1120.500000,207.000000,30.000000,1708.000000,2011-06-24 00:00:00,0.990000,1.0,0.990000
75%,1680.250000,311.000000,45.000000,2560.000000,2012-09-28 00:00:00,0.990000,1.0,0.990000
max,2240.000000,412.000000,59.000000,3500.000000,2013-12-22 00:00:00,1.990000,1.0,1.990000
std,646.776623,119.134877,17.018715,993.797999,NaN,0.217069,0.0,0.217069


In [10]:
# Numerical data summary
print("=== NUMERICAL DATA SUMMARY (fact_sales) ===")
print(f"Date Range: {fact_sales['invoice_date'].min()} to {fact_sales['invoice_date'].max()}")
print(f"Total Number of Sales Transactions: {len(fact_sales):,}")
print(f"Total Revenue: ${fact_sales['total_sale'].sum():.2f}")
print(f"Average Order Value (AOV): ${fact_sales['total_sale'].mean():.2f}")
print(f"Highest Single Sale: ${fact_sales['total_sale'].max():.2f}")
print("\n" + "-"*50 + "\n")

# Categorical data summary
print("=== CATEGORICAL DATA SUMMARY (dim_customers) ===")
print(f"Total Unique Customers: {dim_customers['CustomerId'].nunique()}")
print(f"\nTop 5 Countries by Customer Count:")
print(dim_customers['Country'].value_counts().head())
print(f"\nNumber of Countries: {dim_customers['Country'].nunique()}")

=== NUMERICAL DATA SUMMARY (fact_sales) ===
Date Range: 2009-01-01 00:00:00 to 2013-12-22 00:00:00
Total Number of Sales Transactions: 2,240
Total Revenue: $2328.60
Average Order Value (AOV): $1.04
Highest Single Sale: $1.99

--------------------------------------------------

=== CATEGORICAL DATA SUMMARY (dim_customers) ===
Total Unique Customers: 59

Top 5 Countries by Customer Count:
Country
USA        13
Canada      8
France      5
Brazil      5
Germany     4
Name: count, dtype: int64

Number of Countries: 24


In [11]:
fact_sales.head()

,sales_id,invoice_id,customer_id,track_id,invoice_date,country,unit_price,quantity,total_sale
0,1,1,2,2,2009-01-01,Germany,0.99,1,0.99
1,2,1,2,4,2009-01-01,Germany,0.99,1,0.99
2,3,2,4,6,2009-01-02,Norway,0.99,1,0.99
3,4,2,4,8,2009-01-02,Norway,0.99,1,0.99
4,5,2,4,10,2009-01-02,Norway,0.99,1,0.99


In [12]:
dim_customers.head()

,CustomerId,customer_name,Company,City,Country,SupportRepId,support_rep_name
0,1,Luís Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,São José dos Campos,Brazil,3,Jane Peacock
1,2,Leonie Köhler,None,Stuttgart,Germany,5,Steve Johnson
2,3,François Tremblay,None,Montréal,Canada,3,Jane Peacock
3,4,Bjørn Hansen,None,Oslo,Norway,4,Margaret Park
4,5,František Wichterlová,JetBrains s.r.o.,Prague,Czech Republic,4,Margaret Park


In [14]:
dim_tracks.head()

,TrackId,track_name,album_title,artist_name,genre_name,track_price
0,1,For Those About To Rock (We Salute You),For Those About To Rock We Salute You,AC/DC,Rock,0.99
1,2,Balls to the Wall,Balls to the Wall,Accept,Rock,0.99
2,3,Fast As a Shark,Restless and Wild,Accept,Rock,0.99
3,4,Restless and Wild,Restless and Wild,Accept,Rock,0.99
4,5,Princess of the Dawn,Restless and Wild,Accept,Rock,0.99


<h3>Loading the data to a new analytical database.</h3>

In [15]:
# Loading Data into a New Analytical Database
# Create a new SQLite engine for the transformed data
analytical_engine = create_engine('sqlite:///chinook_analytical.db')

# Load our new tables into the new database
fact_sales.to_sql('fact_sales', analytical_engine, index=False, if_exists='replace')
dim_tracks.to_sql('dim_tracks', analytical_engine, index=False, if_exists='replace')
dim_customers.to_sql('dim_customers', analytical_engine, index=False, if_exists='replace')

print("Data successfully loaded!")

Data successfully loaded!


<h3>Exporting data as csv for Power BI dashboard</h3>

In [16]:
# Data export to CSV for Power BI
fact_sales.to_csv('fact_sales.csv', index=False)
dim_tracks.to_csv('dim_tracks.csv', index=False)
dim_customers.to_csv('dim_customers.csv', index=False)

print("CSV files exported successfully!")

CSV files exported successfully!
